# Libs

In [1]:
import os
import warnings
import pprint
warnings.filterwarnings('ignore')

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

from graph_representation import *

# Create Grid Network

In [ ]:
def create_grid_network(size, show_plot=False):
    '''
    Create a square grid network 
    size: network shape --> (size, size)
    '''

    G = nx.DiGraph()
    
    grid_size = (size,size)

    node_list = []
    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            node_list.append((i,j))
    
    num_bd_nodes = 2 * size + 2 * size
    for i in range(size):
        node_list.append(('nb',i))
        node_list.append(('sb',i))
        node_list.append((i,'wb'))
        node_list.append((i,'eb'))
    
    # add nodes
    for node in node_list:
        G.add_node(node)

    # add internal edges
    max_idx = size - 1
    for node in node_list:
        row, col = node
        if row > 0:
            distance = 300 + 700 * np.random.rand()
            G.add_edge(node,(row - 1,col),distance=distance)
            G.add_edge((row - 1,col),node,distance=distance)
        if col > 0:
            distance = 300 + 700 * np.random.rand()
            G.add_edge(node,(row,col - 1),distance=distance)
            G.add_edge((row,col - 1),node,distance=distance)
        if row < max_idx:
            distance = 300 + 700 * np.random.rand()
            G.add_edge(node,(row + 1,col),distance=distance)
            G.add_edge((row + 1,col),node,distance=distance)
        if col < max_idx:
            distance = 300 + 700 * np.random.rand()
            G.add_edge(node,(row,col + 1),distance=distance)
            G.add_edge((row,col + 1),node,distance=distance)

    # network vis
    if show_plot:
        nx.draw(G,node_size=100,node_color='b',edge_color='k')
        plt.show()
    
    return G

# Transform into Movement Network

In [ ]:
def cal_direction(edge_idx):
    '''
    edge_idx: [row_diff, col_diff]
    # [1,0] --> heading South
    # [-1,0] --> heading North
    # [0,1] --> heading East
    # [0,-1] --> heading West
    '''
    if edge_idx == [1,0]:
        return 'South'
    elif edge_idx == [-1,0]:
        return 'North'
    elif edge_idx == [0,1]:
        return 'East'
    elif edge_idx == [0,-1]:
        return 'West'
    else:
        print('Wrong edge index!')

In [ ]:
margin_edges = []
for row in range(max_idx + 1):
    margin_edges.append(((row, -1),(row, 0)))
    margin_edges.append(((row,max_idx + 1),(row,max_idx)))
for col in range(max_idx + 1):
    margin_edges.append(((-1,col),(0,col)))
    margin_edges.append(((max_idx + 1,col),(max_idx,col)))

In [ ]:
G_mv = nx.DiGraph()
direction_to_mv_origin = {'nbd': {'South': [-1, 0, 1], 'East': [0, 1], 'West': [-1, 0]},
                          'sbd': {'North': [-1, 0, 1], 'East': [0, -1], 'West': [0, 1]},
                          'wbd': {'South': [-1, 0], 'North': [0, 1], 'East': [-1, 0, 1]},
                          'ebd': {'South': [1, 0], 'North': [0, -1], 'West': [-1, 0, 1]},
                          'nwbd': {'South': [0, -1], 'East': [0, 1]},
                          'nebd': {'South': [0, 1], 'West': [-1, 0]},
                          'swbd': {'North': [0, 1], 'East': [0, -1]},
                          'sebd': {'North': [0, -1], 'West': [0, 1]}}
direction_to_mv_destination = {'nwbd': {'North': [1], 'West': [-1]},
                               'nebd': {'North': [-1], 'East': [1]},
                               'swbd': {'South': [-1], 'West': [1]},
                               'sebd': {'South': [1], 'East': [-1]}}

for new_node in list(G.edges) + margin_edges:
    for mv in [-1,0,1]:
        G_mv.add_node(tuple(list(new_node) + [mv]))

In [ ]:
fn_row_minus = lambda a:(a[0] - 1,a[1])
fn_row_plus = lambda a:(a[0] + 1,a[1])
fn_col_minus = lambda a:(a[0],a[1] - 1)
fn_col_plus = lambda a:(a[0],a[1] + 1)

In [ ]:
for node in G_mv.nodes:
    
    direction = cal_direction([node[1][i] - node[0][i] for i in range(2)])
    
    # South, -1 --> col + 1; South, 0 --> row + 1; South, 1 --> col - 1
    # North, -1 --> col - 1; North, 0 --> row - 1; North, 1 --> col + 1
    # East, -1 --> row - 1; East, 0 --> col + 1; East, 1 --> row + 1
    # West, -1 --> row + 1; West, 0 --> col - 1; West, 1 --> row - 1

    from_node = node[1]
    if (direction,node[-1]) in [('South',0),('East',1),('West',-1)]:
        to_node = fn_row_plus(node[1])
    elif (direction,node[-1]) in [('North',0),('East',-1),('West',1)]:
        to_node = fn_row_minus(node[1])
    elif (direction,node[-1]) in [('South',-1),('North',1),('East',0)]:
        to_node = fn_col_plus(node[1])
    elif (direction,node[-1]) in [('South',1),('North',-1),('West',0)]:
        to_node = fn_col_minus(node[1])

    if to_node not in G.nodes:
        continue
    else:
        connecting_node = []
        for mv in [-1,0,1]:
            trial_node = tuple([from_node, to_node] + [mv])
            if trial_node in G_mv.nodes:
                connecting_node.append(trial_node)
        for i in range(len(connecting_node)):
            G_mv.add_edge(node,connecting_node[i],weight=0.1*np.random.rand())

In [ ]:
degree_dict = dict(G_mv.degree)
rmv_node = []
for key in degree_dict.keys():
    if degree_dict[key] == 0:
        rmv_node.append(key)
        G_mv.remove_node(key)

In [ ]:
plt.figure(figsize=(8,8))
nx.draw(G_mv,node_size=30,node_color='r',edge_color='k',pos=nx.kamada_kawai_layout(G_mv))

# Lib Test

In [ ]:
from graph_representation import *

In [ ]:
node_dim = np.zeros((9,2))
edge_dim = np.zeros((9,2))
for size in range(2,11):
    G = create_grid_network(size)
    G_mv = movement_network_transform(G)
    node_dim[size - 2,:] = [len(G.nodes),len(G_mv.nodes)]
    edge_dim[size - 2,:] = [len(G.edges),len(G_mv.edges)]

In [ ]:
plt.rc('font',family='Times New Roman',size=12)
plt.plot(node_dim[:,0],node_dim[:,1],marker='o',c='b')
plt.xlabel('Grid Network Node Number',fontdict={'size':14})
plt.ylabel('Movement Network Node Number',fontdict={'size':14})
k = (node_dim[:,1] / node_dim[:,0]).mean()
plt.text(40,800,'k = {:.2f}'.format(k),fontdict={'color':'b','size':15,'style':'italic'})

In [ ]:
plt.rc('font',family='Times New Roman',size=12)
plt.plot(edge_dim[:,0],edge_dim[:,1],marker='^',c='r')
plt.xlabel('Grid Network Edge Number',fontdict={'size':14})
plt.ylabel('Movement Network Edge Number',fontdict={'size':14})
k = (edge_dim[:,1] / edge_dim[:,0]).mean()
plt.text(150,2000,'k = {:.2f}'.format(k),fontdict={'color':'r','size':15,'style':'italic'})

# Load Path Flow

In [2]:
G = create_grid_network(size=3)
G_mv = movement_network_transform(G)

In [3]:
# select od
def select_od_pairs(G, num):
    """
    Select a portion of Boundary nodes in G.
    Params: G, symmetric grid network
    Params: num, number of selected o/d node
    """
    # init node set
    nodeSet = list(G.nodes)

    # get boundary nodes
    boundNode = []
    maxNode = np.sqrt(len(G.nodes)) - 1
    for idx, (u, v) in enumerate(nodeSet):
        if (u == maxNode) or (v == maxNode) or (u == 0) or (v == 0):
            boundNode.append((u,v))
    
    # select o/d node
    odNode = []
    idx = []
    if num > len(boundNode):
        print('[Error] Given param "num" is too large.')
    
    while len(idx) < num:
        node = np.random.choice(range(len(boundNode)))
        if node not in idx:
            idx.append(node)
    odNode = [boundNode[int(i)] for i in idx]
    
    return odNode

In [4]:
def generate_od_demand_pattern(odNodeSet, mean):
    """
    Generate daily OD demand.
    Params: odNodeSet, set of O/D nodes
    Params: mean, mean value of the normal distribution
    """
    odPairSet = [(u, v) for u in odNodeSet for v in odNodeSet if u != v]

    odDemand = {}
    for idx, pair in enumerate(odPairSet):
        odDemand[pair] = int(np.abs(np.random.normal(mean,mean/3)))
    return odPairSet, odDemand

In [5]:
def get_cost(G, path):
    """
    Get path cost in a weighted network G.
    """
    cost = 0
    for idx, o_node in enumerate(path[:-1]):
        d_node = path[idx + 1]
        edgeCost = G.get_edge_data(o_node, d_node)["distance"]
        cost += edgeCost
    return cost


def generate_path_set(G, o_node, d_node, k, speed_limit=30):
    """
    Get k shortest paths for o_node to d_node in network G.
    """
    pathSet = {}
    for idx, path in enumerate(
        nx.shortest_simple_paths(G, o_node, d_node, weight="distance")
    ):
        if idx < k:
            cost = get_cost(G, path) / (speed_limit / 3.6)
            pathSet[idx] = {"path": path, "cost": cost}
        else:
            break
    return pathSet

In [6]:
odNodeSet = select_od_pairs(G,4)
odPairSet, odDemand = generate_od_demand_pattern(odNodeSet, mean=800)

In [7]:
odPathInfo = {}
for u,v in odPairSet:
    pathSet = generate_path_set(G, u, v, k=2)
    pathCosts = np.array([pathSet[i]['cost'] for i in pathSet.keys()])
    pathProb = np.exp(-pathCosts / 3600) / np.exp(-pathCosts / 3600).sum()
    pathFlow = np.round(odDemand[(u,v)] * pathProb)
    for i in pathSet.keys():
        pathSet[i]['flow'] = pathFlow[i]
    odPathInfo[(u,v)] = pathSet

In [ ]:
# select shortest path

# generate path flow

# generate movement flow (node weight)

# generate movement pair flow (edge weight)